# Escasez de Agua y el Impacto en el Valor de las Propiedades en Ciudad de México

* Author : Axel Daniel Malváez Flores
* Date : 2025-06-26
* Motivación : Análisis que tiene como principal objetivo estudiar la relación entre la escasez de agua en Ciudad de México y el impacto en el valor de las propiedades inmobiliarias. La escasez de agua ya no es solo un problema ambiental, sino un riesgo financiero y urbano tangible. El real estate, como sector clave en el desarrollo económico y el uso del suelo, debe adaptarse a este nuevo paradigma, integrando métricas de sostenibilidad hídrica en sus decisiones de inversión, construcción y planeación.


#### Notas

* Notebook to explore, create plots, analyze data, create data sets and draw conclusions.
* A Streamlit app to visualize the data and findings will be created separately.

## Libraries

In [1]:
# pip install pykrige
# pip install rapidfuzz

In [2]:
# Data Management
import pandas as pd
import numpy as np
import json

# Treemap visualization
import squarify
from shapely.geometry import Point
import seaborn as sns
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import plotly.express as px # Interactive

# To make spatial data
from scipy.spatial import cKDTree
import geopandas as gpd

# No accents
import unicodedata

# Provide a running estimate
from tqdm import tqdm

# Filter out warnings from outputs
import warnings
warnings.filterwarnings('ignore')

## Data

### Water

In [3]:
# Data consumption during the whole 2019 by neighborhood
data_consumo_19 = pd.read_csv('../raw-data/water/consumo/consumo_agua_historico_2019.csv')
data_consumo_19.drop_duplicates(inplace=True)

# Municipalities with drought data and higher/lower probability of drought
data_drought = pd.read_excel('../raw-data/water/sequia/Municipios_con_ sequia.xlsx', sheet_name='MONITOR - SEMAFORO - USO EFIC')
cleaned_cols = data_drought.columns.to_series().where(~data_drought.columns.str.contains('^Unnamed'), '')
data_drought.columns = [x + '/' + y if x != '' else y for x, y in zip(cleaned_cols, data_drought.iloc[0].astype(str))]
data_drought = data_drought.iloc[1:].reset_index().iloc[:,1:]

# Water-related reports for 2022-2024
data_water_reps_22_24 = pd.read_csv('../raw-data/water/reportes/reportes_agua_2024_01.csv')#, parse_dates=['fecha_referencia'])
data_water_reps_hist = pd.read_csv('../raw-data/water/reportes/reportes_agua_hist.csv')#, parse_dates=['fecha_referencia'])

# Hidric feasibility
with open("../raw-data/water/factibilidad/factibilidad-hdrica.json", "r", encoding="utf-8") as f:
    data = json.load(f)
data_feasibility = gpd.GeoDataFrame.from_features(data["features"])

# Consumption every two months by neighborhood
with open('../raw-data/water/consumo/consumo-habitacional-promedio-bimestral-de-agua-por-colonia-m3.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
hab_consumption = gpd.GeoDataFrame.from_features(data['features'])

### Property Prices & Population Statistics

In [4]:
# ------------
# Descriptive
# ------------
data_idx_SHF = pd.read_csv('../raw-data/property-population/SHF/indice_SHF.csv', encoding='iso-8859-1', delimiter=';')
data_tasa_crecimiento_alcaldia = pd.read_csv('../raw-data/property-population/CrecimientoPoblacional/poblacion_total_tasa_crecimiento_alcaldia_1.2.csv', encoding='utf-8')
data_hogares_col = gpd.read_file("../raw-data/property-population/Hogares por colonia/hogares_colonia.shp")
# data_leed = pd.read_excel('data/property-population/Leed/PublicLEEDProjectDirectory.xlsx')

# -------------
# GeoJson Only
# -------------
# Concentración habitacional
data_concentracion = gpd.read_file("../raw-data/property-population/alta_concentracion/zonas_vivienda.shp")

# Rangos del valor del metro cuadrado en la ciudad de méxico
# data_zonas_valor = gpd.read_file("data/property-population/vus_promedio/valores_unitarios.shp")

## Water EDA

### Consumo de agua en Ciudad de México 2019

Información bimestral por el concepto de suministro de agua a nivel manzana, considerando la facturación por servicio de consumo medido y promedio.

***Consideraré únicamente consumos totales, debido a que el promedio no concuerda con el número de viviendas (inmuebles) en las zonas con los otros data sets como el consumo habitacional y el de concentración habitacional que indican que hay mayor número de inmuebles y que estos dos últimos si coinciden entre sí***

**Unidades**   
Medidas expresadas en  m³ de manera bimestral

**Descripción**
Para comenzar a desarrollar una historia podemos comenzar con el consumo de agua en Ciudad de México en 2019. Este es un punto de partida para entender la situación del agua en la ciudad y cómo ha evolucionado a lo largo del tiempo, sin necesidad de entrar en detalles técnicos complejos aún. Con estos datos podemos entender en el 2019 : 

* Cuáles eran las colonias con mayor consumo de agua, y cómo esto puede afectar el valor de las propiedades en esas áreas. 
* Qué tan desarrolladas son esas colonias y cómo influye en el consumo de agua.
* Qué tipo de inmuebles predominan en esas colonias y cómo esto puede relacionarse con el consumo de agua.

**Argumento**
* Refleja la presión real sobre la disponibilidad del recurso.
* La escasez hídrica ocurre por la suma de todos los usos: doméstico, comercial, industrial, institucional, etc.
* Algunas zonas con bajo consumo doméstico pueden estar tensionadas por altos consumos industriales y por lo tanto afectar el valor de las propiedades en esas áreas o si no, entender la afectación a futuro.

#### Preguntas


**Principal**:
* ¿Cuáles son las colonias con mayor y menor consumo de agua? - Usar total agregando por colonia y alcaldía
    * ¿Cuál es el la proporción del índice de desarrollo en dichas colonias?
    * ¿Cuál es el la proporción de la fuente del consumo (i.e. doméstico, no doméstico, mixto)?
    * ¿Hay colonias con un consumo desproporcionado en relación con el número de inmuebles? ¿Cuáles? - Usar total agregando por colonia y alcaldía

**Secundarias**:
* ¿Qué alcaldías tienen el mayor y menor consumo de agua? - Usar total agregando por colonia y alcaldía
    * ¿Cuál es el la proporción del índice de desarrollo en dichas alcaldías?
    * ¿Cuál es el la proporción de la fuente del consumo (i.e. doméstico, no doméstico, mixto)?
    * ¿Hay alcaldías con un consumo desproporcionado en relación con el número de inmuebles? - Usar total agregando por colonia y alcaldía

**External Data Cross-Referencing**:    
* ¿Hay colonias con un consumo desproporcionado en relación a su población?
* ¿Cómo se relaciona el consumo de agua con el valor de las propiedades en esas colonias

#### Data Processing

* Cleaning, feature engineering, and formatting

In [ ]:
data_consumo_19[['fecha_referencia', 'anio', 'bimestre', 'indice_des', 'colonia', 'alcaldia', 'latitud', 'longitud']].drop_duplicates(inplace=True)

In [ ]:
# Date 
data_consumo_19['fecha_referencia'] = pd.to_datetime(data_consumo_19['fecha_referencia'], format='ISO8601')

# Adding derived columns for total consumption
data_consumo_19['inmuebles_domesticos'] = data_consumo_19['consumo_total_dom'] / data_consumo_19['consumo_prom_dom']
data_consumo_19['inmuebles_no_domesticos'] = data_consumo_19['consumo_total_no_dom'] / data_consumo_19['consumo_prom_no_dom']
data_consumo_19['inmuebles_mixtos'] = data_consumo_19['consumo_total_mixto'] / data_consumo_19['consumo_prom_mixto']

# replace NaN values with 0 in the derived column
data_consumo_19['inmuebles_domesticos'].fillna(0, inplace=True)
data_consumo_19['inmuebles_no_domesticos'].fillna(0, inplace=True)
data_consumo_19['inmuebles_mixtos'].fillna(0, inplace=True)

# Data structure and column ordering
data_consumo_19 = data_consumo_19[['fecha_referencia', 'consumo_total', 'inmuebles_domesticos', 'consumo_total_dom', 'consumo_prom_dom', 'inmuebles_no_domesticos', 'consumo_total_no_dom', 'consumo_prom_no_dom', 'inmuebles_mixtos', 'consumo_total_mixto', 'consumo_prom_mixto', 'indice_des', 'colonia', 'alcaldia']]

# Adding total consumption and total number of properties
data_consumo_19['total_inmuebles'] = data_consumo_19['inmuebles_domesticos'] + data_consumo_19['inmuebles_no_domesticos'] + data_consumo_19['inmuebles_mixtos']


# Important : Filter the data to only include the last date of 2019
print("Dates in the dataset:", data_consumo_19['fecha_referencia'].unique())
data_consumo_19_f = data_consumo_19[data_consumo_19['fecha_referencia'] == '2019-06-30']

#### First Aggregation

In [ ]:
# Neighborhood aggregation (since we have duplicated neighborhoods, we can aggregate the data by summing the total consumption)
data_consumo_19_agg = data_consumo_19_f.groupby(['colonia', 'alcaldia']).agg({
    'consumo_total': 'sum',
    'inmuebles_domesticos': 'sum',
    'consumo_total_dom': 'sum',
    'inmuebles_no_domesticos': 'sum',
    'consumo_total_no_dom': 'sum',
    'inmuebles_mixtos': 'sum',
    'consumo_total_mixto': 'sum',
    'total_inmuebles': 'sum'
}).reset_index()

# We filter out rows where all consumption values are zero
data_consumo_19_agg = data_consumo_19_agg[~(data_consumo_19_agg.iloc[:, 2:] == 0).all(axis=1)]

# insert a pivot and create new columns based on indice_des column using the values
share_idx_des = data_consumo_19_f.pivot_table(
    index=['colonia', 'alcaldia'],
    columns='indice_des',
    values='total_inmuebles',
    aggfunc='sum'
).reset_index()

all_agg = data_consumo_19_agg.merge(share_idx_des, on=['colonia', 'alcaldia'], how='left')
all_agg.fillna(0, inplace=True)
all_agg.head()

**Principal**:

* ¿Cuáles son las colonias con mayor y menor consumo de agua? - Usar total agregando por colonia y alcaldía
    * Sin filtros
    * Filtrando por colonias con inmuebles domésticos representando más del 50% del consumo total de agua
    * Filtrando por colonias con inmuebles no domésticos representando más del 50% del consumo total de agua
    * Filtrando por colonias con inmuebles domésticos representando más del 50% del número de inmuebles, pero inmuebles no domésticos representando más del 50% del consumo total de agua - Colonias donde comprar una casa puede representar una posible escasez a futuro

* Sin Filtros

In [ ]:
nf_h = all_agg.sort_values(by='consumo_total', ascending=False)

# ---------------------------
#        MATPLOTLIB
# ---------------------------

# d = nf_h.head(20).copy()

# labels = [
#     f"{row['colonia']},\n{row['alcaldia']}\n({int(row['consumo_total'])} m³)"
#     for _, row in d.iterrows()
# ]
# sizes = d['consumo_total']

# plt.figure(figsize=(16, 9))
# squarify.plot(sizes=sizes, label=labels, alpha=0.8, color=sns.color_palette("viridis", len(sizes)), text_kwargs={'fontsize': 11, 'color': 'black'})
# plt.title("Top 20 Colonias con Mayor Consumo Total de Agua (m³)")
# plt.axis('off')
# plt.show()


# ---------------------------
#          PLOTLY
# ---------------------------


# Prepare the data
d = nf_h.head(20).copy()

# Optional: format the custom label
d['label'] = d.apply(lambda row: f"{row['colonia']},<br>{row['alcaldia']}<br>({int(row['consumo_total'])} m³)", axis=1)

# Plotly treemap
fig = px.treemap(
    d,
    path=['alcaldia', 'colonia'],  # hierarchy levels
    values='consumo_total',
    hover_data={'consumo_total': True},
    color='consumo_total',
    color_continuous_scale='viridis'
)

fig.update_traces(textinfo="label+value")
fig.update_layout(
    title="Top 20 Colonias con Mayor Consumo Total de Agua (m³)",
    title_font_size=20,
    margin=dict(t=50, l=25, r=25, b=25)
)
fig.show()

In [ ]:
# ---------------------------
#        MATPLOTLIB
# ---------------------------

# # Make a pie chart using the proportions in the columns ALTO, BAJO, MEDIO and POPULAR
# plt.figure(figsize=(10, 6))
# sizes = [d['ALTO'].sum(), d['BAJO'].sum(), d['MEDIO'].sum(), d['POPULAR'].sum()]
# labels = ['ALTO', 'BAJO', 'MEDIO', 'POPULAR']
# plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140, colors=sns.color_palette("pastel", len(sizes)))
# plt.title("Proporción del Índice de Desarrollo Urbano (IDU) de las Top 20 colonias más consumidoras")
# plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
# plt.show()

# ---------------------------
#          PLOTLY
# ---------------------------

# Prepare data
sizes = [d['ALTO'].sum(), d['BAJO'].sum(), d['MEDIO'].sum(), d['POPULAR'].sum()]
labels = ['ALTO', 'BAJO', 'MEDIO', 'POPULAR']

# Create a DataFrame
df_pie = pd.DataFrame({'IDU': labels, 'Proporcion': sizes})

# Plotly pie chart
fig = px.pie(
    df_pie,
    names='IDU',
    values='Proporcion',
    title="Proporción del Índice de Desarrollo Urbano (IDU) de las Top 20 colonias más consumidoras",
    color_discrete_sequence=px.colors.qualitative.Pastel
)

fig.update_traces(textinfo='percent+label')  # Show percentage and label
fig.update_layout(
    title_font_size=18,
    title_x=0.5,  # Center title
    height=600,
    width=850
)
fig.show()

In [ ]:
# ---------------------------
#        MATPLOTLIB
# ---------------------------

# # Make a pie chart using the proportions in the columns ALTO, BAJO, MEDIO and POPULAR
# plt.figure(figsize=(10, 6))
# sizes = [d['inmuebles_domesticos'].sum(), d['inmuebles_no_domesticos'].sum(), d['inmuebles_mixtos'].sum()]
# labels = ['inmuebles_domesticos', 'inmuebles_no_domesticos', 'inmuebles_mixtos']
# plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140, colors=sns.color_palette("pastel", len(sizes)))
# plt.title("Proporción del tipo de inmuebles de las Top 20 colonias más consumidoras")
# plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
# plt.show()

# ---------------------------
#          PLOTLY
# ---------------------------
# Preparar datos
sizes = [
    d['inmuebles_domesticos'].sum(),
    d['inmuebles_no_domesticos'].sum(),
    d['inmuebles_mixtos'].sum()
]
labels = ['Inmuebles Domésticos', 'Inmuebles No Domésticos', 'Inmuebles Mixtos']

# Crear DataFrame
df_pie = pd.DataFrame({
    'Tipo de Inmueble': labels,
    'Cantidad': sizes
})

# Gráfico de pastel en Plotly
fig = px.pie(
    df_pie,
    names='Tipo de Inmueble',
    values='Cantidad',
    title="Proporción del tipo de inmuebles de las Top 20 colonias más consumidoras",
    color_discrete_sequence=px.colors.qualitative.Pastel
)

fig.update_traces(textinfo='percent+label')
fig.update_layout(
    title_font_size=18,
    title_x=0.5,  # Centrar título
    height=600,
    width=850
)

fig.show()

* Top 10 Colonias con Menor Consumo Total de Agua (m³)

In [ ]:
# nf_t = all_agg.sort_values(by='consumo_total', ascending=True)

# d = nf_t.head(20).copy()

# labels = [
#     f"{row['colonia']},\n{row['alcaldia']}\n({int(row['consumo_total'])} m³)"
#     for _, row in d.iterrows()
# ]
# sizes = d['consumo_total']

# plt.figure(figsize=(16, 9))
# squarify.plot(sizes=sizes, label=labels, alpha=0.8, color=sns.color_palette("viridis", len(sizes)), text_kwargs={'fontsize': 8, 'color': 'black'})
# plt.title("Top 10 Colonias con Menor Consumo Total de Agua (m³)")
# plt.axis('off')
# plt.show()

In [ ]:
# # Make a pie chart using the proportions in the columns ALTO, BAJO, MEDIO and POPULAR
# plt.figure(figsize=(10, 6))
# sizes = [d['ALTO'].sum(), d['BAJO'].sum(), d['MEDIO'].sum(), d['POPULAR'].sum()]
# labels = ['ALTO', 'BAJO', 'MEDIO', 'POPULAR']
# plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140, colors=sns.color_palette("pastel", len(sizes)))
# plt.title("Proporción de Colonias por Índice de Desarrollo Urbano (IDU) Top 20 menos consumidoras")
# plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
# plt.show()

* Top 10 Colonias con Inmuebles Domésticos que representan más del 50% de Consumo Total de Agua (m³)

In [ ]:
# yf_dom_h = all_agg[all_agg.iloc[:,2:]['consumo_total_dom'] / all_agg.iloc[:,2:]['consumo_total'] > 0.5].sort_values(by='consumo_total', ascending=False)
# d = yf_dom_h.head(20).copy()

# labels = [
#     f"{row['colonia']},\n{row['alcaldia']}\n({int(row['consumo_total'])} m³)"
#     for _, row in d.iterrows()
# ]
# sizes = d['consumo_total']

# plt.figure(figsize=(16, 9))
# squarify.plot(sizes=sizes, label=labels, alpha=0.8, color=sns.color_palette("viridis", len(sizes)), text_kwargs={'fontsize': 11, 'color': 'black'})
# plt.title("Top 10 Colonias con Inmuebles Domésticos que representan más del 50% de Consumo Total de Agua (m³)")
# plt.axis('off')
# plt.show()

In [ ]:
# # Make a pie chart using the proportions in the columns ALTO, BAJO, MEDIO and POPULAR
# plt.figure(figsize=(10, 6))
# sizes = [d['ALTO'].sum(), d['BAJO'].sum(), d['MEDIO'].sum(), d['POPULAR'].sum()]
# labels = ['ALTO', 'BAJO', 'MEDIO', 'POPULAR']
# plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140, colors=sns.color_palette("pastel", len(sizes)))
# plt.title("Proporción de Colonias por Índice de Desarrollo Urbano (IDU) Inmuebles Domésticos que representan más del 50% de Consumo Total de Agua (m³)")
# plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
# plt.show()

* Filtrando por alcaldías con inmuebles no domésticos representando más del 50% del consumo total de agua

In [ ]:
yf_no_dom_h = all_agg[all_agg.iloc[:,2:]['consumo_total_no_dom'] / all_agg.iloc[:,2:]['consumo_total'] > 0.5].sort_values(by='consumo_total', ascending=False)

d = yf_no_dom_h.head(20).copy()
labels = [
    f"{row['colonia']},\n{row['alcaldia']}\n({int(row['consumo_total'])} m³)"
    for _, row in d.iterrows()
]
sizes = d['consumo_total']

plt.figure(figsize=(16, 9))
squarify.plot(sizes=sizes, label=labels, alpha=0.8, color=sns.color_palette("viridis", len(sizes)), text_kwargs={'fontsize': 11, 'color': 'black'})
plt.title("Top 10 Colonias con Inmuebles No Domésticos que representan más del 50% de Consumo Total de Agua (m³)")
plt.axis('off')
plt.show()

In [ ]:
# Make a pie chart using the proportions in the columns ALTO, BAJO, MEDIO and POPULAR
plt.figure(figsize=(10, 6))
sizes = [d['ALTO'].sum(), d['BAJO'].sum(), d['MEDIO'].sum(), d['POPULAR'].sum()]
labels = ['ALTO', 'BAJO', 'MEDIO', 'POPULAR']
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140, colors=sns.color_palette("pastel", len(sizes)))
plt.title("Proporción de Colonias por Índice de Desarrollo Urbano (IDU) Inmuebles No Domésticos que representan más del 50% de Consumo Total de Agua (m³)")
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.show()

* Filtrando por alcaldías con inmuebles domésticos representando más del 50% del número de inmuebles, pero inmuebles no domésticos representando más del 50% del consumo total de agua

In [ ]:
# yf_dom_no_h = all_agg[(all_agg.iloc[:,2:]['inmuebles_domesticos'] / all_agg.iloc[:,2:]['total_inmuebles'] > 0.8) & 
#                     (all_agg.iloc[:,2:]['consumo_total_no_dom'] / all_agg.iloc[:,2:]['consumo_total'] > 0.6)].sort_values(by='consumo_total', ascending=False)

In [ ]:
# d = yf_dom_no_h.head(20).copy()

# labels = [
#     f"{row['colonia']},\n{row['alcaldia']}\n({int(row['consumo_total'])} m³)"
#     for _, row in d.iterrows()
# ]
# sizes = d['consumo_total']

# plt.figure(figsize=(16, 9))
# squarify.plot(sizes=sizes, label=labels, alpha=0.8, color=sns.color_palette("viridis", len(sizes)), text_kwargs={'fontsize': 8, 'color': 'black'})
# plt.title("Top 10 Colonias con más del 50% Inmuebles Dom e Inmuebles No Dom que representan más del 50% de Consumo Total de Agua (m³)")
# plt.axis('off')
# plt.show()

In [ ]:
# # Make a pie chart using the proportions in the columns ALTO, BAJO, MEDIO and POPULAR
# plt.figure(figsize=(10, 6))
# sizes = [d['ALTO'].sum(), d['BAJO'].sum(), d['MEDIO'].sum(), d['POPULAR'].sum()]
# labels = ['ALTO', 'BAJO', 'MEDIO', 'POPULAR']
# plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140, colors=sns.color_palette("pastel", len(sizes)))
# plt.title("Proporción de Colonias por Índice de Desarrollo Urbano (IDU) Top 10 con más del 50% Inmuebles Dom e Inmuebles No Dom que representan más del 50% de Consumo Total de Agua (m³)")
# plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
# plt.show()

* Existe una relación entre el índice de desarrollo y el consumo de agua en las colonias, donde las colonias con un índice de desarrollo más bajo tienden a tener un mayor consumo de agua or menor consumo de agua, y viceversa. 

In [ ]:
# Correlación entre tipo de inmueble y consumo doméstico
correlation_matrix = all_agg[['ALTO', 'BAJO', 'MEDIO', 'POPULAR', 'consumo_total']].corr()

# Heatmap
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title("Matriz de Correlación")
plt.show()

In [ ]:
all_agg['mayoria_idx'] = all_agg[['ALTO', 'BAJO', 'MEDIO', 'POPULAR']].idxmax(axis=1)
dict1 = {
    'ALTO': 'blue',
    'MEDIO': 'green',
    'BAJO': 'orange',
    'POPULAR': 'red'
    }

all_agg['mayoria_type'] = all_agg[['inmuebles_domesticos', 'inmuebles_no_domesticos', 'inmuebles_mixtos']].idxmax(axis=1)

dict2 = {'inmuebles_domesticos' : 'blue',
         'inmuebles_no_domesticos' : 'green',
         'inmuebles_mixtos' : 'orange'}

all_agg['consumo_mayoria'] =  all_agg[['consumo_total_dom', 'consumo_total_no_dom', 'consumo_total_mixto']].idxmax(axis=1)
dict3 = {'consumo_total_dom' : 'blue',
         'consumo_total_no_dom' : 'green',
         'consumo_total_mixto' : 'orange'}

In [ ]:
# plt.figure(figsize=(10, 6))

# for category, color in dict1.items():
#     subset = all_agg[all_agg['mayoria_idx'] == category]
#     plt.scatter(subset['total_inmuebles'], subset['consumo_total'],
#                 alpha=0.7, c=color, label=f'Índice de Desarrollo {category}')


# plt.xlabel('Total de Inmuebles')
# plt.ylabel('Consumo Total de Agua (m³)')
# plt.title('Relación entre Total de Inmuebles y Consumo Total de Agua por Colonia')
# plt.grid(True)
# plt.legend()
# plt.show()


# Make sure your category column is categorical
all_agg['mayoria_idx'] = all_agg['mayoria_idx'].astype(str)

# Create the scatterplot
fig = px.scatter(
    all_agg,
    x='total_inmuebles',
    y='consumo_total',
    color='mayoria_idx',
    color_discrete_map=dict1,  # Apply your custom color mapping
    labels={
        'total_inmuebles': 'Total de Inmuebles',
        'consumo_total': 'Consumo Total de Agua (m³)',
        'mayoria_idx': 'Índice de Desarrollo'
    },
    title='Relación entre Total de Inmuebles y Consumo Total de Agua por Colonia',
    opacity=0.7
)

fig.update_layout(
    title_font_size=18,
    title_x=0.5,
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    height=900,
    width=1000
)

fig.show()

In [ ]:
# fig, axes = plt.subplots(1, 4, figsize=(30, 7))  # Increase figsize for better spacing

# category = 'ALTO'
# subset_alto = data_consumo_19_f[data_consumo_19_f['indice_des'] == category]
# axes[0].scatter(subset_alto['total_inmuebles'], subset_alto['consumo_total'], alpha=0.7, label=f'Índice de Desarrollo {category}')
# axes[0].set_title('Total Inmuebles Desarrollo Alto vs Consumo agua (m³)')
# axes[0].set_ylim(0, 130000)

# category = 'MEDIO'
# subset_medio = data_consumo_19_f[data_consumo_19_f['indice_des'] == category]
# axes[1].scatter(subset_medio['total_inmuebles'], subset_medio['consumo_total'], alpha=0.7, label=f'Índice de Desarrollo {category}')
# axes[1].set_title('Total Inmuebles Desarrollo Medio vs Consumo agua (m³)')
# axes[1].set_ylim(0, 130000)

# category = 'BAJO'
# subset_bajo = data_consumo_19_f[data_consumo_19_f['indice_des'] == category]
# axes[2].scatter(subset_bajo['total_inmuebles'], subset_bajo['consumo_total'], alpha=0.7, label=f'Índice de Desarrollo {category}')
# axes[2].set_title('Total Inmuebles Desarrollo Bajo vs Consumo agua (m³)')
# axes[2].set_ylim(0, 130000)

# category = 'POPULAR'
# subset_pop = data_consumo_19_f[data_consumo_19_f['indice_des'] == category]
# axes[3].scatter(subset_pop['total_inmuebles'], subset_pop['consumo_total'], alpha=0.7, label=f'Índice de Desarrollo {category}')
# axes[3].set_title('Total Inmuebles Desarrollo Popular vs Consumo agua (m³)')
# axes[3].set_ylim(0, 130000)

# plt.show()

In [ ]:
print(f"El consumo promedio de todo tipo de inmuebles con un índice de desarrollo ALTO en CDMX : {data_consumo_19_f[data_consumo_19_f['indice_des'] == 'ALTO']['consumo_total'].sum() / data_consumo_19_f[data_consumo_19_f['indice_des'] == 'ALTO']['total_inmuebles'].sum()}")
print(f"El consumo promedio de todo tipo de inmuebles con un índice de desarrollo MEDIO en CDMX : {data_consumo_19_f[data_consumo_19_f['indice_des'] == 'MEDIO']['consumo_total'].sum() / data_consumo_19_f[data_consumo_19_f['indice_des'] == 'MEDIO']['total_inmuebles'].sum()}")
print(f"El consumo promedio de todo tipo de inmuebles con un índice de desarrollo BAJO en CDMX : {data_consumo_19_f[data_consumo_19_f['indice_des'] == 'BAJO']['consumo_total'].sum() / data_consumo_19_f[data_consumo_19_f['indice_des'] == 'BAJO']['total_inmuebles'].sum()}")
print(f"El consumo promedio de todo tipo de inmuebles con un índice de desarrollo POPULAR en CDMX : {data_consumo_19_f[data_consumo_19_f['indice_des'] == 'POPULAR']['consumo_total'].sum() / data_consumo_19_f[data_consumo_19_f['indice_des'] == 'POPULAR']['total_inmuebles'].sum()}")

In [ ]:
# fig, axes = plt.subplots(1, 4, figsize=(30, 7))  # Increase figsize for better spacing

# category = 'ALTO'
# subset_alto = data_consumo_19_f[data_consumo_19_f['indice_des'] == category]
# total_cat = data_consumo_19_f[data_consumo_19_f['indice_des'] == 'ALTO']['inmuebles_domesticos'].sum()
# axes[0].scatter(subset_alto['inmuebles_domesticos'], subset_alto['consumo_total_dom'], alpha=0.7, label=f'Índice de Desarrollo {category}')
# axes[0].set_title(f'Consumo vs # Inmuebles Dom con Alto IDU' + f' ({int(total_cat)} inmuebles)')
# axes[0].set_ylim(0, 100000)
# axes[0].set_xlabel('Total Inmuebles')
# axes[0].set_ylabel('Consumo de Agua (m³)')

# category = 'MEDIO'
# subset_medio = data_consumo_19_f[data_consumo_19_f['indice_des'] == category]
# total_cat = data_consumo_19_f[data_consumo_19_f['indice_des'] == 'MEDIO']['inmuebles_domesticos'].sum()
# axes[1].scatter(subset_medio['inmuebles_domesticos'], subset_medio['consumo_total_dom'], alpha=0.7, label=f'Índice de Desarrollo {category}')
# axes[1].set_title(f'Consumo vs # Inmuebles Dom con Medio IDU' + f' ({int(total_cat)} inmuebles)')
# axes[1].set_ylim(0, 100000)
# axes[1].set_xlabel('Total Inmuebles')
# axes[1].set_ylabel('Consumo de Agua (m³)')

# category = 'BAJO'
# subset_bajo = data_consumo_19_f[data_consumo_19_f['indice_des'] == category]
# total_cat = data_consumo_19_f[data_consumo_19_f['indice_des'] == 'BAJO']['inmuebles_domesticos'].sum()
# axes[2].scatter(subset_bajo['inmuebles_domesticos'], subset_bajo['consumo_total_dom'], alpha=0.7, label=f'Índice de Desarrollo {category}')
# axes[2].set_title(f'Consumo vs # Inmuebles Dom con Bajo IDU' + f' ({int(total_cat)} inmuebles)')
# axes[2].set_ylim(0, 100000)
# axes[2].set_xlabel('Total Inmuebles')
# axes[2].set_ylabel('Consumo de Agua (m³)')

# category = 'POPULAR'
# subset_pop = data_consumo_19_f[data_consumo_19_f['indice_des'] == category]
# total_cat = data_consumo_19_f[data_consumo_19_f['indice_des'] == 'POPULAR']['inmuebles_domesticos'].sum()
# axes[3].scatter(subset_pop['inmuebles_domesticos'], subset_pop['consumo_total_dom'], alpha=0.7, label=f'Índice de Desarrollo {category}')
# axes[3].set_title(f'Consumo vs # Inmuebles Dom con Popular IDU' + f' ({int(total_cat)} inmuebles)')
# axes[3].set_ylim(0, 100000)
# axes[3].set_xlabel('Total Inmuebles')
# axes[3].set_ylabel('Consumo de Agua (m³)')

# plt.show()

In [ ]:
print(f"El consumo promedio de casas con un índice de desarrollo ALTO en CDMX : {data_consumo_19_f[data_consumo_19_f['indice_des'] == 'ALTO']['consumo_total_dom'].sum() / data_consumo_19_f[data_consumo_19_f['indice_des'] == 'ALTO']['inmuebles_domesticos'].sum()}")
print(f"El número total de inmuebles habitacionales con índice de desarrollo ALTO en CDMX : {data_consumo_19_f[data_consumo_19_f['indice_des'] == 'ALTO']['inmuebles_domesticos'].sum()}")
print(f"El número total de métros cúbicos consumidos por inmuebles con índice de desarrollo ALTO en CDMX : {data_consumo_19_f[data_consumo_19_f['indice_des'] == 'ALTO']['consumo_total_dom'].sum()}")
print()

print(f"El consumo promedio de casas con un índice de desarrollo MEDIO en CDMX : {data_consumo_19_f[data_consumo_19_f['indice_des'] == 'MEDIO']['consumo_total_dom'].sum() / data_consumo_19_f[data_consumo_19_f['indice_des'] == 'MEDIO']['inmuebles_domesticos'].sum()}")
print(f"El número total de inmuebles habitacionales con índice de desarrollo MEDIO en CDMX : {data_consumo_19_f[data_consumo_19_f['indice_des'] == 'MEDIO']['inmuebles_domesticos'].sum()}")
print(f"El número total de métros cúbicos consumidos por inmuebles con índice de desarrollo MEDIO en CDMX : {data_consumo_19_f[data_consumo_19_f['indice_des'] == 'MEDIO']['consumo_total_dom'].sum()}")
print()

print(f"El consumo promedio de casas con un índice de desarrollo BAJO en CDMX : {data_consumo_19_f[data_consumo_19_f['indice_des'] == 'BAJO']['consumo_total_dom'].sum() / data_consumo_19_f[data_consumo_19_f['indice_des'] == 'BAJO']['inmuebles_domesticos'].sum()}")
print(f"El número total de inmuebles habitacionales con índice de desarrollo BAJO en CDMX : {data_consumo_19_f[data_consumo_19_f['indice_des'] == 'BAJO']['inmuebles_domesticos'].sum()}")
print(f"El número total de métros cúbicos consumidos por inmuebles con índice de desarrollo BAJO en CDMX : {data_consumo_19_f[data_consumo_19_f['indice_des'] == 'BAJO']['consumo_total_dom'].sum()}")
print()

print(f"El consumo promedio de casas con un índice de desarrollo POPULAR en CDMX : {data_consumo_19_f[data_consumo_19_f['indice_des'] == 'POPULAR']['consumo_total_dom'].sum() / data_consumo_19_f[data_consumo_19_f['indice_des'] == 'POPULAR']['inmuebles_domesticos'].sum()}")
print(f"El número total de inmuebles habitacionales con índice de desarrollo POPULAR en CDMX : {data_consumo_19_f[data_consumo_19_f['indice_des'] == 'POPULAR']['inmuebles_domesticos'].sum()}")
print(f"El número total de métros cúbicos consumidos por inmuebles con índice de desarrollo POPULAR en CDMX : {data_consumo_19_f[data_consumo_19_f['indice_des'] == 'POPULAR']['consumo_total_dom'].sum()}")
print()

In [ ]:
# data_consumo_19_f[data_consumo_19_f['consumo_total_dom'] > 40000].sort_values(by='consumo_total_dom', ascending=False)

In [ ]:
# fig, axes = plt.subplots(1, 4, figsize=(30, 7))  # Increase figsize for better spacing

# category = 'ALTO'
# subset_alto = data_consumo_19_f[data_consumo_19_f['indice_des'] == category]
# total_cat = data_consumo_19_f[data_consumo_19_f['indice_des'] == 'ALTO']['inmuebles_no_domesticos'].sum()
# axes[0].scatter(subset_alto['inmuebles_no_domesticos'], subset_alto['consumo_total_no_dom'], alpha=0.7, label=f'Índice de Desarrollo {category}')
# axes[0].set_title(f'Consumo vs # Inmuebles No Dom con Alto IDU' + f' ({int(total_cat)} inmuebles)')
# axes[0].set_ylim(0, 100000)
# axes[0].set_xlabel('Total Inmuebles')
# axes[0].set_ylabel('Consumo de Agua (m³)')

# category = 'MEDIO'
# subset_medio = data_consumo_19_f[data_consumo_19_f['indice_des'] == category]
# total_cat = data_consumo_19_f[data_consumo_19_f['indice_des'] == 'MEDIO']['inmuebles_no_domesticos'].sum()
# axes[1].scatter(subset_medio['inmuebles_no_domesticos'], subset_medio['consumo_total_no_dom'], alpha=0.7, label=f'Índice de Desarrollo {category}')
# axes[1].set_title(f'Consumo vs # Inmuebles No Dom con Medio IDU' + f' ({int(total_cat)} inmuebles)')
# axes[1].set_ylim(0, 100000)
# axes[1].set_xlabel('Total Inmuebles')
# axes[1].set_ylabel('Consumo de Agua (m³)')

# category = 'BAJO'
# subset_bajo = data_consumo_19_f[data_consumo_19_f['indice_des'] == category]
# total_cat = data_consumo_19_f[data_consumo_19_f['indice_des'] == 'BAJO']['inmuebles_no_domesticos'].sum()
# axes[2].scatter(subset_bajo['inmuebles_no_domesticos'], subset_bajo['consumo_total_no_dom'], alpha=0.7, label=f'Índice de Desarrollo {category}')
# axes[2].set_title(f'Consumo vs # Inmuebles No Dom con Bajo IDU' + f' ({int(total_cat)} inmuebles)')
# axes[2].set_ylim(0, 100000)
# axes[2].set_xlabel('Total Inmuebles')
# axes[2].set_ylabel('Consumo de Agua (m³)')

# category = 'POPULAR'
# subset_pop = data_consumo_19_f[data_consumo_19_f['indice_des'] == category]
# total_cat = data_consumo_19_f[data_consumo_19_f['indice_des'] == 'POPULAR']['inmuebles_no_domesticos'].sum()
# axes[3].scatter(subset_pop['inmuebles_no_domesticos'], subset_pop['consumo_total_no_dom'], alpha=0.7, label=f'Índice de Desarrollo {category}')
# axes[3].set_title(f'Consumo vs # Inmuebles No Dom con Popular IDU' + f' ({int(total_cat)} inmuebles)')
# axes[3].set_ylim(0, 100000)
# axes[3].set_xlabel('Total Inmuebles')
# axes[3].set_ylabel('Consumo de Agua (m³)')

plt.show()

In [ ]:
# data_consumo_19_f[data_consumo_19_f['consumo_total_no_dom'] > 40000].sort_values(by='consumo_total_no_dom', ascending=False)

* ¿Cuál es el la proporción de la fuente del consumo (i.e. doméstico, no doméstico, mixto)?

In [ ]:
# Make a pie chart using the proportions in the columns ALTO, BAJO, MEDIO and POPULAR
plt.figure(figsize=(10, 6))

sizes = [data_consumo_19_f['consumo_total_dom'].sum(), data_consumo_19_f['consumo_total_no_dom'].sum(), data_consumo_19_f['consumo_total_mixto'].sum()]
labels = ['consumo_total_dom', 'consumo_total_no_dom', 'consumo_total_mixto']

plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140, colors=sns.color_palette("pastel", len(sizes)))
plt.title("Proporción de la Fuente del Consumo")
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.show()

* ¿Hay colonias con un consumo desproporcionado en relación con el número de inmuebles? ¿Cuáles? - Usar total agregando por colonia y alcaldía

In [ ]:
# Podemos encontrar ese consumo desproporcionado utilizando el consumo promedio
consumo_prom = data_consumo_19_f[['consumo_prom_dom', 'consumo_prom_no_dom', 'consumo_prom_mixto', 'indice_des', 'colonia', 'alcaldia']]
consumo_prom.sort_values(by='consumo_prom_dom', ascending=False).head(20)

* Cuántas colonias representan más del 50% del consumo total doméstico

In [ ]:
suma = 0
i = 0
while suma < all_agg['consumo_total_dom'].sum() * 0.5: 
    suma += all_agg.sort_values(by='consumo_total_dom', ascending=False)['consumo_total_dom'][i]
    i +=1
    
i

* Alcaldías que más gastan

In [ ]:
# Neighborhood aggregation (since we have duplicated neighborhoods, we can aggregate the data by summing the total consumption)
data_consumo_19_agg_alc = data_consumo_19_f.groupby(['alcaldia']).agg({
    'consumo_total': 'sum',
    'inmuebles_domesticos': 'sum',
    'consumo_total_dom': 'sum',
    #'consumo_prom_dom': 'mean',
    'inmuebles_no_domesticos': 'sum',
    'consumo_total_no_dom': 'sum',
    #'consumo_prom_no_dom': 'mean',
    'inmuebles_mixtos': 'sum',
    'consumo_total_mixto': 'sum',
    'total_inmuebles': 'sum',
    #'indice_des': 'mean'
}).reset_index()


# We filter out rows where all consumption values are zero
data_consumo_19_agg_alc = data_consumo_19_agg_alc[~(data_consumo_19_agg_alc.iloc[:, 2:] == 0).all(axis=1)]

# insert a pivot and create new columns based on indice_des column using the values
share_idx_des_alc = data_consumo_19_f.pivot_table(
    index=['alcaldia'],
    columns='indice_des',
    values='total_inmuebles',
    aggfunc='sum'
).reset_index()

all_agg_alc = data_consumo_19_agg_alc.merge(share_idx_des_alc, on=['alcaldia'], how='left')
all_agg_alc.fillna(0, inplace=True)

# Alcaldias con mayor consumo total
all_agg_alc.sort_values(by='consumo_total', ascending=False)

In [ ]:
# Alcaldias con mayor consumo doméstico
all_agg_alc.sort_values(by='consumo_total_dom', ascending=False)

In [ ]:
# Alcaldias con mayor consumo no doméstico
all_agg_alc.sort_values(by='consumo_total_no_dom', ascending=False)

* ¿Cuál es el la proporción del índice de desarrollo en la Ciudad de México?

In [ ]:
# En toda la ciudad
plt.figure(figsize=(150,10))
sizes = [all_agg_alc['ALTO'].sum(), all_agg_alc['MEDIO'].sum(), all_agg_alc['BAJO'].sum(), all_agg_alc['POPULAR'].sum()]
labels = ['ALTO', 'MEDIO', 'BAJO', 'POPULAR']
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=0, colors=sns.color_palette("pastel", len(sizes)))
plt.title('Proporción de, Índice de Desarrollo en la Ciudad de México de todo tipo de inmuebles')
plt.show()

* ¿Cuál es el la proporción de la fuente del consumo (i.e. doméstico, no doméstico, mixto)?

In [ ]:
plt.figure(figsize=(15,10))
sizes = [all_agg_alc['consumo_total_dom'].sum(), all_agg_alc['consumo_total_no_dom'].sum(), all_agg_alc['consumo_total_mixto'].sum()]
labels = ['Consumo Doméstico', 'Consumo No Doméstico', 'Consumo Mixto']
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=0, colors=sns.color_palette('pastel', len(sizes)))
plt.title('Proporción de la Fuente de Consumo de agua en CDMX')
plt.show()

* Alcaldías con un consumo desproporcionado

In [ ]:
all_agg_alc['consumo_dom_prom'] = all_agg_alc['consumo_total_dom'] / all_agg_alc['inmuebles_domesticos']
all_agg_alc['consumo_no_dom_prom'] = all_agg_alc['consumo_total_no_dom'] / all_agg_alc['inmuebles_no_domesticos']
all_agg_alc['consumo_mixto_prom'] = all_agg_alc['consumo_total_mixto'] / all_agg_alc['inmuebles_mixtos']

In [ ]:
all_agg_alc.sort_values(by='consumo_dom_prom', ascending=False)

### Factibilidad Hídrica en Ciudad de México (2016)


| Grado de Factibilidad Hídrica | Descripción / Significado                                | Color       
| ----------------------------- | -------------------------------------------------------- | ----------- 
| Alta factibilidad 🌟          | Zonas con alta capacidad para mitigar riesgo de escasez  | 🟩 Verde    
| Media-alta factibilidad ⚠️    | Zonas con buena capacidad, pero con algunas limitaciones | 🟨 Amarillo 
| Media-baja factibilidad 🔶    | Zonas con capacidad limitada para mitigar riesgo         | 🟧 Naranja  
| Baja factibilidad ❌           | Zonas con poca o nula capacidad para mitigar riesgo      | 🟥 Rojo    

In [ ]:
# ---------------------------
#          PLOTLY
# ---------------------------

data_feasibility = data_feasibility.iloc[:-1,:]
data_feasibility.rename(columns={'NOMBRE':'colonia', 'DELEGACIO' : 'alcaldia'}, inplace=True)

data_feasibility = gpd.GeoDataFrame(data_feasibility)
data_feasibility = data_feasibility.set_crs(epsg=4326, inplace=True)

# Mapea los colores
dir_color = {
    'ROJO': 'red',
    'AMARILLO': 'yellow',
    'NARANJA': 'orange',
    'VERDE': 'green'
}
data_feasibility['color'] = data_feasibility['fact_hidr'].map(dir_color)

data_feasibility = data_feasibility.reset_index(drop=True)
data_feasibility['id'] = data_feasibility.index

fig = px.choropleth_mapbox(
    data_feasibility,
    geojson=data_feasibility.__geo_interface__,
    locations='id',
    featureidkey='properties.id',
    color='fact_hidr',
    hover_name="colonia",                # columna principal que aparece en negrita
    hover_data={
        "alcaldia": True,                  # otra columna, ej. población total
        "colonia": True,          # densidad de vivienda
        "fact_hidr" : True
    },
    color_discrete_map=dir_color,
    mapbox_style="carto-positron",
    center={"lat": 19.4, "lon": -99.13},
    zoom=10,
    opacity=0.8
)

fig.update_layout(
    margin={"r": 0, "t": 30, "l": 0, "b": 0},
    title="Factibilidad Hídrica en Ciudad de México",
    title_font_size=18,
    height=900,
    width=900
)

fig.show()

### Consumo Habitacional (2019)

Diccionario : 

**SUM_cons_t**   
Suma del consumo total de agua por colonia (m3) por bimestre

**MEAN_cons_**   
Promedio de consumo total de colonia (m3) por bimestre

**VIV2010**   
Vivienda censo 2010

**PROMVIVCON**   
"Promedio de consumo total de agua de la colonia por número de viviendas"

**C_PROMVIVC**   
"Campo reclasificado en cuantiles en 5 rangos a partir de "PROMVIVCON"; donde el valor "5" corresponde a un consumo de agua muy alto, mientras que el número "1" a un consumo bajo"



In [ ]:
# Create a Plotly choropleth_mapbox
fig = px.choropleth_mapbox(
    hab_consumption,
    geojson=hab_consumption.__geo_interface__,      # GeoJSON representation
    locations=hab_consumption.index,                # Use the GeoDataFrame index
    color="C_PROMVIVC",                             # Column to color by
    hover_name="colonia",                # columna principal que aparece en negrita
    hover_data={
        "C_PROMVIVC": True,                 # sigue mostrando consumo
        "alcaldia": True,                  # otra columna, ej. población total
        "colonia": True,          # densidad de vivienda
        "SUM_cons_t" : True
    },
    color_continuous_scale="viridis",               # Viridis color scale
    mapbox_style="carto-positron",                  # Map style
    zoom=10,                                        # Zoom level
    center={"lat": 19.3333, "lon": -99.1333},       # Centered on CDMX
    opacity=0.7,                                    # Fill opacity
    labels={"C_PROMVIVC": "Consumo Promedio"}       # Legend label
)

# Update layout with title and margins
fig.update_layout(
    title_text="Consumo de Agua en Ciudad de México",
    title_font_size=14,
    margin={"l":0, "r":0, "t":30, "b":0},
    height = 900,
    width=900
)

fig.show()

* [Deprecated] Consumo de agua en las alcaldías de la Ciudad de México por fuente de consumo.

*Deprecated por el hecho de que los números entre los datasets en que se hacen merge no son consistentes. La principal causa es que el total de consumo es muy similar a lo que se dice en un data set pero el total de inmuebles no lo es*

In [ ]:
# Bulding a classification depending on total consumption of each county
all_agg['q_class_all'] = pd.qcut(all_agg['consumo_total'], q=5, labels=False)
all_agg['q_class_all'] = all_agg['q_class_all'] + 1

all_agg['q_class_dom'] = pd.qcut(all_agg['consumo_total_dom'], q=5, labels=False)
all_agg['q_class_dom'] = all_agg['q_class_dom'] + 1

# mergin two dfs
data_hab = hab_consumption[['geometry', 'alcaldia', 'colonia', 'PROMVIVCON', 'C_PROMVIVC', 'SUM_cons_t']].merge(all_agg, left_on=['alcaldia','colonia'], right_on=['alcaldia', 'colonia'], how='left')
data_hab.loc[data_hab['q_class_dom'].isna(), 'q_class_dom'] = data_hab.loc[data_hab['q_class_dom'].isna(), 'C_PROMVIVC']
data_hab.loc[data_hab['q_class_all'].isna(), 'q_class_all'] = data_hab.loc[data_hab['q_class_all'].isna(), 'C_PROMVIVC']
data_hab.loc[data_hab['consumo_total'].isna(), 'consumo_total'] = data_hab.loc[data_hab['consumo_total'].isna(), 'SUM_cons_t']

data_hab_agg = data_hab.groupby('alcaldia').agg({
    'consumo_total' : 'sum',
    'consumo_total_dom' : 'sum',
    'consumo_total_no_dom' : 'sum',
     'consumo_total_mixto' : 'sum',
}).reset_index()
data_hab_agg.sort_values(by='consumo_total', inplace=True)

# Set positions for the bars
alcaldias = data_hab_agg['alcaldia']
dom = data_hab_agg['consumo_total_dom']
no_dom = data_hab_agg['consumo_total_no_dom']
mixto = data_hab_agg['consumo_total_mixto']

# Create stacked bar plot
fig, ax = plt.subplots(figsize=(10, 6))

ax.barh(alcaldias, dom, label='Doméstico')
ax.barh(alcaldias, no_dom, left=dom, label='No Doméstico')
ax.barh(alcaldias, mixto, left=dom + no_dom, label='Mixto')

# Labels and legend
ax.set_ylabel('Consumo Total')
ax.set_title('Consumo de Agua por Alcaldía y Tipo de Uso')
ax.legend()

# plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### Probabilidad de Escasez

In [5]:
# Load CDMX alcaldías GeoJSON
with open('../raw-data/water/sequia/limite-de-las-alcaldias.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
alcaldias_cdmx = gpd.GeoDataFrame.from_features(data['features'])
alcaldias_cdmx = alcaldias_cdmx.set_crs(epsg=4326, inplace=True)
geojson = alcaldias_cdmx.__geo_interface__

# Step 1: Identify id_vars (the static columns)
id_vars = ['Region de lluvias y fecha de solicitud de corroboración', 'CVE_CONCATENADA', 'CVE_ENT', 'CVE_MUN', 'NOMBRE_MUN', 'ENTIDAD',
           'ORG_CUENCA', 'CLV_OC', 'CON_CUENCA', 'CVE_CONC']

# Step 2: Melt all other columns
df_melted = data_drought.melt(id_vars=id_vars, var_name='variable', value_name='value')

# Create category and date columns
category = []
date = []
for i in df_melted['variable'].astype(str):
    tup = i.split('/')
    category.append(tup[0])
    date.append(tup[1])
    
df_melted['CATEGORY'] = category
df_melted['CATEGORY'] = df_melted['CATEGORY'].str.replace(r'\.\d+$', '', regex=True)

df_melted['DATE'] = date
df_melted['DATE'] = pd.to_datetime(df_melted['DATE'])
df_melted['MONTH'] = df_melted['DATE'].dt.month_name()
df_melted['YEAR'] = df_melted['DATE'].dt.year
df_melted['DATE'] = df_melted['DATE'].dt.strftime('%Y-%m')

# Mexico city filter, adding map and taking MAGNITUD value onl7
mxc_drought = df_melted[(df_melted['ENTIDAD'] == 'Ciudad de México') & (df_melted['CATEGORY'] == 'MAGNITUD')]
mxc_drought = pd.merge(mxc_drought, alcaldias_cdmx[['CVE_MUN', 'geometry']], on='CVE_MUN', how='left')

color_map = {
    'SIN SEQUIA': '#f0f0f0',     # Very light gray / near white
    'PRE-ALERTA': '#1a9850',     # Light gray
    'VERDE': '#a6d96a',          # Soft green
    'AMARILLO': '#ffffbf',       # Yellow
    'NARANJA': '#fdae61',        # Orange
    'ROJO': '#d73027'            # Red
}

mxc_drought['color'] = mxc_drought[mxc_drought['CATEGORY'] == 'MAGNITUD']['value'].map(color_map)
mxc_drought = gpd.GeoDataFrame(mxc_drought, geometry='geometry')
# mxc_drought["geometry"] = mxc_drought["geometry"].simplify(tolerance=0.0005, preserve_topology=True)

# Data Processing
number_map = {
    'SIN SEQUIA': 1,     # Very light gray / near white
    'PRE-ALERTA': 2,     # Light gray
    'VERDE': 3,          # Soft green
    'AMARILLO': 4,       # Yellow
    'NARANJA': 5,        # Orange
    'ROJO': 6            # Red
}

mxc_drought['VALUE'] = mxc_drought[mxc_drought['CATEGORY'] == 'MAGNITUD']['value'].map(number_map)
mxc_drought.head()

,Region de lluvias y fecha de solicitud de corroboración,CVE_CONCATENADA,CVE_ENT,CVE_MUN,NOMBRE_MUN,ENTIDAD,ORG_CUENCA,CLV_OC,CON_CUENCA,CVE_CONC,variable,value,CATEGORY,DATE,MONTH,YEAR,geometry,color,VALUE
0,Región A - Primeros Quince días hábiles de cad...,09002,09,002,Azcapotzalco,Ciudad de México,Aguas del Valle de México,XIII,Valle de Mexico,26,MAGNITUD/2003-01-31 00:00:00,PRE-ALERTA,MAGNITUD,2003-01,January,2003,"POLYGON ((-99.18231 19.50748, -99.18229 19.507...",#1a9850,2
1,Región A - Primeros Quince días hábiles de cad...,09003,09,003,Coyoacán,Ciudad de México,Aguas del Valle de México,XIII,Valle de Mexico,26,MAGNITUD/2003-01-31 00:00:00,PRE-ALERTA,MAGNITUD,2003-01,January,2003,"POLYGON ((-99.13427 19.35654, -99.13397 19.356...",#1a9850,2
2,Región A - Primeros Quince días hábiles de cad...,09004,09,004,Cuajimalpa de Morelos,Ciudad de México,Aguas del Valle de México,XIII,Valle de Mexico,26,MAGNITUD/2003-01-31 00:00:00,PRE-ALERTA,MAGNITUD,2003-01,January,2003,"POLYGON ((-99.25738 19.40112, -99.25698 19.400...",#1a9850,2
3,Región A - Primeros Quince días hábiles de cad...,09005,09,005,Gustavo A. Madero,Ciudad de México,Aguas del Valle de México,XIII,Valle de Mexico,26,MAGNITUD/2003-01-31 00:00:00,PRE-ALERTA,MAGNITUD,2003-01,January,2003,"POLYGON ((-99.11124 19.56150, -99.11485 19.557...",#1a9850,2
4,Región A - Primeros Quince días hábiles de cad...,09006,09,006,Iztacalco,Ciudad de México,Aguas del Valle de México,XIII,Valle de Mexico,26,MAGNITUD/2003-01-31 00:00:00,PRE-ALERTA,MAGNITUD,2003-01,January,2003,"POLYGON ((-99.05751 19.40673, -99.05753 19.406...",#1a9850,2


* Comparison between all 2003, 2013 and 2023 years (Animated plots)

In [ ]:
mxc_drought_2003_s = mxc_drought[mxc_drought['DATE'] == '2003-01-01']
mxc_drought_2013_s = mxc_drought[mxc_drought['DATE'] == '2013-01-01']
mxc_drought_2023_s = mxc_drought[mxc_drought['DATE'] == '2023-01-01']

mxc_drought_2003_a = mxc_drought[(mxc_drought['DATE'] >= '2003-01-01') & (mxc_drought['DATE'] <= '2003-12-01')]
mxc_drought_2013_a = mxc_drought[(mxc_drought['DATE'] >= '2013-01-01') & (mxc_drought['DATE'] <= '2013-12-01')]
mxc_drought_2023_a = mxc_drought[(mxc_drought['DATE'] >= '2023-01-01') & (mxc_drought['DATE'] <= '2023-12-01')]

In [7]:
def plot_static_map(df, title, show=True, write=False, file_name=None) : 
    # Define your color mapping
    color_discrete_map = {
        'SIN SEQUIA': '#f0f0f0',
        'PRE-ALERTA': '#d9d9d9',
        'VERDE': '#a6d96a',
        'AMARILLO': '#ffffbf',
        'NARANJA': '#fdae61',
        'ROJO': '#d73027'
    }

    fig = px.choropleth_mapbox(
        df,
        geojson=df.__geo_interface__,
        locations=df.index,  # You can use index if each row is unique
        color="value",  # Use categorical value, not raw hex codes
        hover_name="NOMBRE_MUN",
        hover_data=["CATEGORY", "value", 'DATE'],
        mapbox_style="carto-positron",
        zoom=9,
        center={"lat": 19.33, "lon": -99.13},
        opacity=0.8,
        color_discrete_map=color_discrete_map
    )


    fig.update_layout(
        margin={"r":0, "t":30, "l":0, "b":0},
        title=title,
        width=1000,    # width in pixels
        height=700,    # height in pixels
        showlegend=False
    )

    if show:
        # Show hover labels directly on the map
        fig.update_traces()
        fig.show()

    if write : 
        if file_name == None : 
            file_name = 'fig.html'
        fig.write_html(file_name)
    
def plot_animated_map(df, title, show=True, write=False, file_name=None) : 
    # Convert DATE to string if it's datetime (required by Plotly animation)
    # df["DATE"] = df["DATE"].astype(str)

    # Define category → color mapping
    color_discrete_map = {
        'SIN SEQUIA': '#f0f0f0',
        'PRE-ALERTA': '#d9d9d9',
        'VERDE': '#a6d96a',
        'AMARILLO': '#ffffbf',
        'NARANJA': '#fdae61',
        'ROJO': '#d73027'
    }
    
    color_map = {
    "1": '#f0f0f0',     # Very light gray / near white
    "2": '#1a9850',     # Light gray
    "3": '#a6d96a',          # Soft green
    "4": '#ffe066',       # Yellow
    "5": '#fdae61',        # Orange
    "6": '#d73027'            # Red
    }
    
    mxc_drought_t = df[['DATE', 'geometry', 'NOMBRE_MUN', 'VALUE', 'color']]
    
    vmin = np.log10(mxc_drought_t["VALUE"].min())
    vmax = np.log10(mxc_drought_t["VALUE"].max())

    def normalize(val):
        return (val - 1) / (6 - 1)

    custom_scale = [
        (normalize(1), "#f0f0f0"),
        (normalize(2), "#1a9850"),
        (normalize(3), "#a6d96a"),
        (normalize(4), "#ffe066"),
        (normalize(5), "#fdae61"),
        (normalize(6), "#d73027"),
    ]    

    fig = px.choropleth_mapbox(mxc_drought_t,
                        geojson=mxc_drought_t.__geo_interface__,
                        locations="NOMBRE_MUN",
                        color=np.log10(mxc_drought_t["VALUE"]),
                        hover_name="NOMBRE_MUN",
                        hover_data=["VALUE"],
                        animation_frame='DATE',
                        featureidkey='properties.NOMBRE_MUN',  # Adjust based on GeoJSON field
                        # color_continuous_midpoint = 1,
                        color_continuous_scale=custom_scale,
                        range_color=[0,1],
                        # color_discrete_map=color_map,
                        mapbox_style="carto-positron",
                        center={"lat": 19.33, "lon": -99.1332},  # Center on Mexico City
                        zoom=9,)

    fig.update_layout(margin=dict(l=20,r=0,b=0,t=70,pad=0),
                    paper_bgcolor="white",
                    width = 1000,
                    height= 700,
                    title_text = 'Evolución de la Sequía en la Ciudad de México 2003-2023',
                    font_size=18,
                    )
    # fig.show()

    if show : 
        # Show hover labels directly on the map
        # fig.update_traces()
        fig.show()
        
    if write : 
        if file_name == None:
            file_name = 'fig.html'
        fig.write_html(file_name)

In [11]:
# --------------
# NOT INSIGHTFUL
# --------------

# plot_static_map(mxc_drought_2003_s, 'Sequía en Ciudad de México en Oct 2003')
# plot_animated_map(mxc_drought_2003_a, 'Sequía en Ciudad de México en el año 2003', show=True, write=False, file_name='sequia_mxc_2003.html')

In [19]:
# plot_static_map(mxc_drought_2013_s, 'Sequía en Ciudad de México en Oct 2013')
# plot_animated_map(mxc_drought_2013_a, 'Sequía en Ciudad de México en el año 2013', show=True, write=False, file_name='sequia_mxc_2013.html')

In [21]:
# plot_static_map(mxc_drought_2023_s, 'Sequía en Ciudad de México en Oct 2023')
# plot_animated_map(mxc_drought_2023_a, 'Sequía en Ciudad de México en el año 2023', show=False, write=True, file_name='sequia_mxc_2023.html')

* Comparison between each year

In [ ]:
# Filtering by year
# mxc_drought_years_only = mxc_drought[(mxc_drought['MONTH'] == 'October')]
# plot_animated_map(mxc_drought, 'Evolución de la Sequía en la Ciudad de México 2003-2023 (Oct only)', show=False, write=True, file_name='sequia_evolucion_mxc_oct.html')

#### Insights

- Entender cuánto ha cambiado con el tiempo el número de alcaldías en alerta con sequía
    * Maybe del 2023 vs T-5 años
    * Maybe del 2023 vs T-10 años
    * Maybe del 2023 vs T-20 años

Cuantitativo y en gráfica

In [ ]:
mxc_drought[mxc_drought['YEAR'] == 2023]['value'].value_counts()

In [ ]:
avg_status_per_year = (mxc_drought[['YEAR', 'CVE_MUN', 'NOMBRE_MUN', 'VALUE']].groupby(by=['YEAR', 'CVE_MUN','NOMBRE_MUN'])['VALUE'].sum()/12).reset_index()
avg_merge = pd.merge(avg_status_per_year, alcaldias_cdmx[['geometry', 'CVE_MUN']], on='CVE_MUN', how='left')
avg_merge = gpd.GeoDataFrame(avg_merge)

# avg_merge['VALUE'] = avg_merge['VALUE']/10

In [ ]:
def normalize(val):
    return (val - 1) / (6 - 1)

custom_scale = [
        (normalize(1), "#f0f0f0"),
        (normalize(2), "#1a9850"),
        (normalize(3), "#a6d96a"),
        (normalize(4), "#ffe066"),
        (normalize(5), "#fdae61"),
        (normalize(6), "#d73027"),
    ]    

In [ ]:
# fig = px.choropleth_mapbox(
#     avg_merge[avg_merge['YEAR'] == 2023],
#     geojson=avg_merge[avg_merge['YEAR'] == 2023].__geo_interface__,
#     locations=avg_merge[avg_merge['YEAR'] == 2023].index,  # You can use index if each row is unique
#     color=[normalize(x) for x in avg_merge[avg_merge['YEAR'] == 2023]['VALUE']],  # Use categorical value, not raw hex codes
#     hover_name="NOMBRE_MUN",
#     hover_data=["VALUE", "NOMBRE_MUN"],
#     color_continuous_scale=custom_scale,
#     range_color=[0,1],
#     mapbox_style="carto-positron",
#     zoom=9,
#     center={"lat": 19.33, "lon": -99.13},
#     opacity=0.8,
# )

# fig.update_layout(
#     margin={"r":0, "t":30, "l":0, "b":0},
#     paper_bgcolor="white",
#     title='Estado de sequía en promedio en 2023',
#     width=1000,    # width in pixels
#     height=700,    # height in pixels
#     showlegend=False,
#     font_size=18
# )

# fig.show()

# file_name = 'sequia_promedio_2023.html'
# fig.write_html(file_name)

In [ ]:
# fig = px.choropleth_mapbox(
#     avg_merge,
#     geojson=avg_merge.__geo_interface__,
#     locations="NOMBRE_MUN",  # You can use index if each row is unique
#     color=[normalize(x) for x in avg_merge['VALUE']],  # Use categorical value, not raw hex codes
#     hover_name="NOMBRE_MUN",
#     hover_data=["VALUE", "NOMBRE_MUN"],
#     color_continuous_scale=custom_scale,
#     animation_frame='YEAR',
#     featureidkey='properties.NOMBRE_MUN',  # Adjust based on GeoJSON field
#     animation_group="NOMBRE_MUN",
#     range_color=[0,1],
#     mapbox_style="carto-positron",
#     zoom=9,
#     center={"lat": 19.33, "lon": -99.13},
#     opacity=0.8,
# )

# fig.update_layout(
#     margin={"r":0, "t":30, "l":0, "b":0},
#     paper_bgcolor="white",
#     title='Estado de sequía en promedio en 2003-2023',
#     width=1000,    # width in pixels
#     height=700,    # height in pixels
#     showlegend=False,
#     font_size=18
# )

# fig.show()

# file_name = 'sequia_promedio_evolution_3.html'
# fig.write_html(file_name)

In [ ]:
mxc_drought[mxc_drought['YEAR'] == 2018]['value'].value_counts()

In [ ]:
# fig = px.choropleth_mapbox(
#     avg_merge[avg_merge['YEAR'] == 2018],
#     geojson=avg_merge[avg_merge['YEAR'] == 2018].__geo_interface__,
#     locations=avg_merge[avg_merge['YEAR'] == 2018].index,  # You can use index if each row is unique
#     color=[normalize(x) for x in avg_merge[avg_merge['YEAR'] == 2018]['VALUE']],  # Use categorical value, not raw hex codes
#     hover_name="NOMBRE_MUN",
#     hover_data=["VALUE", "NOMBRE_MUN"],
#     color_continuous_scale=custom_scale,
#     range_color=[0,1],
#     mapbox_style="carto-positron",
#     zoom=9,
#     center={"lat": 19.33, "lon": -99.13},
#     opacity=0.8,
# )

# fig.update_layout(
#     margin={"r":0, "t":30, "l":0, "b":0},
#     paper_bgcolor="white",
#     title='Estado de sequía en promedio en 2018',
#     width=1000,    # width in pixels
#     height=700,    # height in pixels
#     showlegend=False,
#     font_size=18
# )

# fig.show()

# file_name = 'sequia_promedio_2023.html'
# fig.write_html(file_name)

In [ ]:
mxc_drought[mxc_drought['YEAR'] == 2013]['value'].value_counts()

In [ ]:
# fig = px.choropleth_mapbox(
#     avg_merge[avg_merge['YEAR'] == 2013],
#     geojson=avg_merge[avg_merge['YEAR'] == 2013].__geo_interface__,
#     locations=avg_merge[avg_merge['YEAR'] == 2013].index,  # You can use index if each row is unique
#     color=[normalize(x) for x in avg_merge[avg_merge['YEAR'] == 2013]['VALUE']],  # Use categorical value, not raw hex codes
#     hover_name="NOMBRE_MUN",
#     hover_data=["VALUE", "NOMBRE_MUN"],
#     color_continuous_scale=custom_scale,
#     range_color=[0,1],
#     mapbox_style="carto-positron",
#     zoom=9,
#     center={"lat": 19.33, "lon": -99.13},
#     opacity=0.8,
# )

# fig.update_layout(
#     margin={"r":0, "t":30, "l":0, "b":0},
#     paper_bgcolor="white",
#     title='Estado de sequía en promedio en 2013',
#     width=1000,    # width in pixels
#     height=700,    # height in pixels
#     showlegend=False,
#     font_size=18
# )

# fig.show()

In [ ]:
mxc_drought[mxc_drought['YEAR'] == 2003]['value'].value_counts()

In [ ]:
# fig = px.choropleth_mapbox(
#     avg_merge[avg_merge['YEAR'] == 2003],
#     geojson=avg_merge[avg_merge['YEAR'] == 2003].__geo_interface__,
#     locations=avg_merge[avg_merge['YEAR'] == 2003].index,  # You can use index if each row is unique
#     color=[normalize(x) for x in avg_merge[avg_merge['YEAR'] == 2003]['VALUE']],  # Use categorical value, not raw hex codes
#     hover_name="NOMBRE_MUN",
#     hover_data=["VALUE", "NOMBRE_MUN"],
#     color_continuous_scale=custom_scale,
#     range_color=[0,1],
#     mapbox_style="carto-positron",
#     zoom=9,
#     center={"lat": 19.33, "lon": -99.13},
#     opacity=0.8,
# )

# fig.update_layout(
#     margin={"r":0, "t":30, "l":0, "b":0},
#     paper_bgcolor="white",
#     title='Estado de sequía en promedio en 2003',
#     width=1000,    # width in pixels
#     height=700,    # height in pixels
#     showlegend=False,
#     font_size=18
# )

# fig.show()

* All history 2003-2023

In [ ]:
# Aggregate data
t = mxc_drought.groupby(by=['DATE', 'MONTH', 'YEAR'])['VALUE'].mean().reset_index()
t['DATE'] = pd.to_datetime(t['DATE'])

# Filter date range (2003 to 2023)
t_filtered = t[(t['DATE'] >= "2003-01-01") & (t['DATE'] <= "2023-12-31")]

# Create Plotly line plot
fig = px.line(
    t_filtered,
    x='DATE',
    y='VALUE',
    markers=True,
    labels={'VALUE': 'Drought Category', 'DATE': 'Date'},
    title='Escasez del agua en Ciudad de México (0 - sin escasez, 6 - alta escasez)'
)

# Set y-axis ticks manually
fig.update_yaxes(tickmode='array', tickvals=[1, 2, 3, 4, 5, 6])

# Format x-axis range and tick labels
fig.update_xaxes(
    range=[t_filtered['DATE'].min(), t_filtered['DATE'].max()],
    tickformat="%Y",
    dtick="M12",  # One tick every 12 months
    tickangle=45
)

fig.update_layout(
    width=2400,
    height=650,
    template='plotly_white'
)

fig.show()


### Reportes de Agua

Info : 
- Data ranging from 2018 to 2024

* History of reports
    * The most common report
    * Separate all this by alcaldia and colonia

* Analyze time and identify trends/patterns in which all this reports are made.
    * Tipically after what hour or during which range of hours.

In [ ]:
data_reports_22_24 = data_water_reps_22_24[['fecha_reporte','hora_reporte', 'reporte', 'alcaldia_catalogo', 'colonia_catalogo', 'longitud', 'latitud']]
data_reports_hist_22 = data_water_reps_hist[['fecha','tipo_de_falla', 'alcaldia', 'colonia_datos_abiertos', 'longitud', 'latitud']]

new_names_cols = {'tipo_de_falla' : 'reporte', 
                  'alcaldia_catalogo' : 'alcaldia', 
                  'colonia_catalogo' : 'colonia',
                  'colonia_datos_abiertos' : 'colonia',
                  'fecha_reporte' : 'fecha'
                  }

data_reports_hist_22.rename(columns=new_names_cols, inplace=True)
data_reports_22_24.rename(columns=new_names_cols, inplace=True)

all_reports = pd.concat([data_reports_22_24, data_reports_hist_22], axis=0)

all_reports['fecha'] = pd.to_datetime(all_reports['fecha'])
all_reports['year'] = all_reports['fecha'].dt.year
all_reports['month_name'] = all_reports['fecha'].dt.month_name()
all_reports['month'] = all_reports['fecha'].dt.month

all_reports = all_reports.dropna(subset=['alcaldia', 'colonia'])
all_reports.head()

In [ ]:
# Most common reports per year
reports_count_p_y = all_reports.groupby(by=['year', 'reporte']).size().reset_index(name='report_count')
pivot_yr = reports_count_p_y.pivot_table(index='year', columns='reporte', values='report_count')
pivot_yr.fillna(0, inplace=True)

# Make a copy of your table
df_yr = pivot_yr.copy()
keep = ['Fuga', 'Falta de agua']
df_yr['Otro'] = df_yr.drop(columns=keep).sum(axis=1)

# Keep only desired + 'Otros'
df_filtered_yr = df_yr[keep + ['Otro']]

# Melt the data to long format
df_long = df_filtered_yr.reset_index().melt(id_vars='year', var_name='Tipo de Reporte', value_name='Cantidad')

# Ensure month order
import calendar
# df_long['month_name'] = pd.Categorical(df_long['month_name'], categories=list(calendar.month_name[1:]), ordered=True)

fig = px.bar(
    df_long,
    x='year',
    y='Cantidad',
    color='Tipo de Reporte',
    title='Reportes por Mes: Fuga, Falta de Agua y Otros',
    barmode='stack',
    labels={'year': 'Año'}
)

fig.update_layout(template="plotly_white")
fig.show()

In [ ]:
# Most common reports per month name
reports_count_p_y = all_reports.groupby(by=['month', 'month_name', 'reporte']).size().reset_index(name='report_count')
pivot_mth = reports_count_p_y.pivot_table(index=['month','month_name'], columns='reporte', values='report_count')
pivot_mth

df_mth = pivot_mth.copy()
keep = ['Fuga', 'Falta de agua']
df_mth['Otro'] = df_mth.drop(keep, axis=1).sum(axis=1)
df_filtered_mth = df_mth[keep + ['Otro']]

# Melt the data to long format
df_long = df_filtered_mth.reset_index().melt(id_vars=['month', 'month_name'], var_name='Tipo de Reporte', value_name='Cantidad')

# Ensure month order
import calendar
# df_long['month_name'] = pd.Categorical(df_long['month_name'], categories=list(calendar.month_name[1:]), ordered=True)

fig = px.bar(
    df_long,
    x='month_name',
    y='Cantidad',
    color='Tipo de Reporte',
    title='Reportes por Mes: Fuga, Falta de Agua y Otros',
    barmode='stack',
    labels={'month_name': 'Año'}
)

fig.update_layout(template="plotly_white")
fig.show()

#### Raster Map - Falta de Agua 2022 & 2024

Usaré el método de interpolación IDW (inverse distance weighting - distancia inversa ponderada)

In [ ]:
def idw_interpolation(xy_known, values_known, xy_grid, power=2, k=3):
    """
    xy_known: (N, 2) array of known [lon, lat]
    values_known: (N,) array of known values
    xy_grid: (M, 2) array of grid [lon, lat]
    power: IDW power (2 is common)
    k: number of nearest neighbors to use
    """
    tree = cKDTree(xy_known)
    dists, idxs = tree.query(xy_grid, k=k)
    
    dists[dists == 0] = 1e-10  # avoid division by zero
    weights = 1 / dists**power
    weights /= weights.sum(axis=1, keepdims=True)

    interpolated = np.sum(values_known[idxs] * weights, axis=1)
    return interpolated

In [ ]:
reports_count_p_y_m = all_reports.groupby(by=['year', 'alcaldia', 'colonia', 'latitud', 'longitud', 'reporte']).size().reset_index(name='report_count')
pivot_all = reports_count_p_y_m.pivot_table(index=['year', 'alcaldia', 'colonia', 'latitud', 'longitud'], columns='reporte', values='report_count')

df_all = pivot_all.copy()
keep = ['Fuga', 'Falta de agua']
df_all['Otro'] = df_all.drop(columns=keep).sum(axis=1)

df_all = df_all[keep + ['Otro']]

df_all.reset_index(inplace=True)
df_all.fillna(0, inplace=True)

# Rename for clarity
df_all = df_all.rename(columns={
    'latitud': 'latitude',
    'longitud': 'longitude',
    'Falta de agua': 'falta_agua_count',
    'Fuga': 'fuga_count',
    'Otro': 'otro_count'    
})

# Data Set creation
df_falta_agua_2022 = df_all[(df_all['year'] == 2022)] #& (df_all['falta_agua_count'] > 0)]
df_falta_agua_2024 = df_all[(df_all['year'] == 2024)] #& (df_all['falta_agua_count'] > 0)]

# For mexico city map (neighborhoods included)
temp_copy = data_hab[['geometry', 'alcaldia', 'colonia']]
minx, miny, maxx, maxy = temp_copy.total_bounds

# General Grid to interpolate over
grid_lon = np.linspace(minx, maxx, 200)
grid_lat = np.linspace(miny, maxy, 200)

grid_lon_mesh, grid_lat_mesh = np.meshgrid(grid_lon, grid_lat)
grid_points = np.c_[grid_lon_mesh.ravel(), grid_lat_mesh.ravel()]

* Reporte de falta de agua por colonia (mapa) para 2022

In [ ]:
df_to_use = df_falta_agua_2022
value_to_use = 'falta_agua_count'

# Known points
xy_known = df_to_use[['longitude', 'latitude']].values
z_known = df_to_use[value_to_use].values  # or 'falta de agua'

# Grid Already Created, then we go to interpolation directly
z_idw_flat = idw_interpolation(xy_known, z_known, grid_points, power=0.7, k=50)
z_idw = z_idw_flat.reshape(grid_lat_mesh.shape)

# Flatten the raster grid
grid_df = pd.DataFrame({
    'lon': grid_lon_mesh.ravel(),
    'lat': grid_lat_mesh.ravel(),
    'value': z_idw.ravel()  # or z_kriging.ravel()
})

# Create geometry column
grid_df['geometry'] = [Point(xy) for xy in zip(grid_df['lon'], grid_df['lat'])]
grid_gdf = gpd.GeoDataFrame(grid_df, geometry='geometry', crs="EPSG:4326")

cdmx_union = data_hab.unary_union  # combines all alcaldías into one

# Spatial filter
grid_inside = grid_gdf[grid_gdf.geometry.within(cdmx_union)]

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=grid_inside['lon'],
    y=grid_inside['lat'],
    mode='markers',
    marker=dict(
        size=5,
        color=grid_inside['value'],
        colorscale='YlOrRd',
        colorbar=dict(title='Interpolated Intensity')
    ),
    text=grid_inside['value'].round(2),         # show rounded values
    hoverinfo='text',
    #name='Raster (Clipped)'
))

from shapely.geometry import Polygon, MultiPolygon

# Loop through each geometry (Polygon or MultiPolygon)
for geom in temp_copy['geometry']:
    if geom is None:
        continue
    if isinstance(geom, Polygon):
        x, y = geom.exterior.xy
        fig.add_trace(go.Scatter(
            x=list(x), y=list(y),
            mode='lines',
            line=dict(color='black', width=0.3),
            name='CDMX Boundary',
            showlegend=False
        ))
    elif isinstance(geom, MultiPolygon):
        for poly in geom.geoms:
            x, y = poly.exterior.xy
            fig.add_trace(go.Scatter(
                x=list(x), y=list(y),
                mode='lines',
                line=dict(color='black', width=0.3),
                name='CDMX Boundary',
                showlegend=False
            ))

fig.add_trace(go.Scatter(
    x=list(x), y=list(y),
    mode='lines',
    fill='toself',
    fillcolor='rgba(0,0,0,0.05)',  # light gray fill
    line=dict(color='black', width=1),
    showlegend=False
))


fig.update_layout(
    title='Interpolated Water Reports with IDW',
    xaxis_title='Longitude',
    yaxis_title='Latitude',
    width=900,
    height=1000
)

fig.show()


* Reporte de falta de agua por colonia (mapa) para 2024

In [ ]:
df_to_use = df_falta_agua_2024
value_to_use = 'falta_agua_count'

# Known points
xy_known = df_to_use[['longitude', 'latitude']].values
z_known = df_to_use[value_to_use].values  # or 'falta de agua'

# Grid Already Created, then we go to interpolation directly
z_idw_flat = idw_interpolation(xy_known, z_known, grid_points, power=0.7, k=50)
z_idw = z_idw_flat.reshape(grid_lat_mesh.shape)

# Flatten the raster grid
grid_df = pd.DataFrame({
    'lon': grid_lon_mesh.ravel(),
    'lat': grid_lat_mesh.ravel(),
    'value': z_idw.ravel()  # or z_kriging.ravel()
})

# Create geometry column
grid_df['geometry'] = [Point(xy) for xy in zip(grid_df['lon'], grid_df['lat'])]
grid_gdf = gpd.GeoDataFrame(grid_df, geometry='geometry', crs="EPSG:4326")

cdmx_union = data_hab.unary_union  # combines all alcaldías into one

# Spatial filter
grid_inside = grid_gdf[grid_gdf.geometry.within(cdmx_union)]

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=grid_inside['lon'],
    y=grid_inside['lat'],
    mode='markers',
    marker=dict(
        size=5,
        color=grid_inside['value'],
        colorscale='YlOrRd',
        colorbar=dict(title='Interpolated Intensity')
    ),
    text=grid_inside['value'].round(2),         # show rounded values
    hoverinfo='text',
    #name='Raster (Clipped)'
))

from shapely.geometry import Polygon, MultiPolygon

# Loop through each geometry (Polygon or MultiPolygon)
for geom in temp_copy['geometry']:
    if geom is None:
        continue
    if isinstance(geom, Polygon):
        x, y = geom.exterior.xy
        fig.add_trace(go.Scatter(
            x=list(x), y=list(y),
            mode='lines',
            line=dict(color='black', width=0.3),
            name='CDMX Boundary',
            showlegend=False
        ))
    elif isinstance(geom, MultiPolygon):
        for poly in geom.geoms:
            x, y = poly.exterior.xy
            fig.add_trace(go.Scatter(
                x=list(x), y=list(y),
                mode='lines',
                line=dict(color='black', width=0.3),
                name='CDMX Boundary',
                showlegend=False
            ))

fig.add_trace(go.Scatter(
    x=list(x), y=list(y),
    mode='lines',
    fill='toself',
    fillcolor='rgba(0,0,0,0.05)',  # light gray fill
    line=dict(color='black', width=1),
    showlegend=False
))


fig.update_layout(
    title='Interpolated Water Reports with IDW',
    xaxis_title='Longitude',
    yaxis_title='Latitude',
    width=900,
    height=1000
)

fig.show()


## Population and Property EDA

### Concentración Habitacional

Este dataset es úncamente un GeoJson que muestra en un rastermap la concentración habitacional en la ciudad de México

| Campo         | Nombre                     | Tipo de dato          | Descripción                                                                                      |
|---------------|-----------------------------|------------------------|--------------------------------------------------------------------------------------------------|
| cve_ent       | Clave de la entidad         | Número entero          | Claves de Áreas Geoestadísticas Estatales, de acuerdo al Marco Geoestadístico Nacional para la Ciudad de México |
| alcaldia      | Alcaldía                    | Alfanumérico           | Claves de Áreas Geoestadísticas Municipal, de acuerdo al Marco Geoestadístico Nacional para la Ciudad de México |
| cve_col       | Clave de la colonia         | Alfanumérico           | Claves de las Unidades Territoriales a partir del Instituto Electoral de la Ciudad de México    |
| colonia       | Colonia                     | Alfanumérico           | El nombre debe corresponder con el código del campo cve_col, de conformidad con el Instituto Electoral de la Ciudad de México |
| VivHab2010    | Viviendas habitadas 2010    | Número entero          | Información a partir del Censo de Población y Vivienda 2010                                     |
| VivHab2020    | Viviendas habitadas 2020    | Número entero          | Información a partir del Censo de Población y Vivienda 2020                                     |
| Area_ha       | Área en hectáreas           | Número con decimales   |                                                                                                  |
| DenViv10      | Densidad de viviendas 2010  | Número con decimales   | Información a partir del Censo de Población y Vivienda 2010                                     |
| DenViv20      | Densidad de viviendas 2020  | Número con decimales   | Información a partir del Censo de Población y Vivienda 2020                                     |
| Sum_TotHog    | Suma del total de hogares   | Número entero          | Información a partir del Censo de Población y Vivienda 2020                                     |


In [ ]:
# 1. Ensure both GeoDataFrames use the same coordinate reference system (CRS)
data_hogares_col = data_hogares_col.to_crs(data_concentracion.crs)

# 2. Perform spatial join
hogares_with_grado = gpd.sjoin(
    data_hogares_col,
    data_concentracion[['geometry', 'grado', 'ID']],  # only keep necessary columns
    predicate='intersects',  # or 'intersects' if geometries might overlap partially
    how='left'
)

In [ ]:
fig = go.Figure()

# 1. Plot `data_concentracion` polygons, colored by 'ID' using a colormap
from matplotlib import cm
from matplotlib.colors import Normalize, to_hex

# Normalize and get a color map (e.g., viridis)
norm = Normalize(vmin=data_concentracion['ID'].min(), vmax=data_concentracion['ID'].max())
cmap = cm.get_cmap('viridis')

for _, row in data_concentracion.iterrows():
    geom = row['geometry']
    val = row['ID']
    grado = row['grado']
    color = to_hex(cmap(norm(val)))  # Convert color to hex for Plotly

    if isinstance(geom, Polygon):
        polys = [geom]
    elif isinstance(geom, MultiPolygon):
        polys = geom.geoms
    else:
        continue

    for poly in polys:
        x, y = poly.exterior.xy
        fig.add_trace(go.Scatter(
            x=list(x),
            y=list(y),
            mode='lines',
            line=dict(color=color, width=1),
            fill='toself',
            fillcolor=color,
            name=f'ID {val}\nGrado {grado}',
            hoverinfo='all',
            showlegend=False
        ))

# Add a dummy scatter trace just to show the colorbar
fig.add_trace(go.Scatter(
    x=[None],
    y=[None],
    mode='markers',
    marker=dict(
        colorscale='Viridis',
        cmin=data_concentracion['ID'].min(),
        cmax=data_concentracion['ID'].max(),
        colorbar=dict(
            title='ID',
            thickness=15,
            len=0.75,
            x=1.02,  # Position to the right
            xanchor='left'
        ),
        color=np.linspace(data_concentracion['ID'].min(), data_concentracion['ID'].max(), 1),
        showscale=True
    ),
    hoverinfo='none',
    showlegend=False
))

# 2. Overlay `data_hogares_col` as transparent outlines with hover info
for i, row in hogares_with_grado.iterrows():
    geom = row['geometry']
    name = row['colonia']  # County/Colonia name
    grado = row['grado'] 
    ID = row['ID'] 

    if isinstance(geom, Polygon):
        polys = [geom]
    elif isinstance(geom, MultiPolygon):
        polys = geom.geoms
    else:
        continue

    for poly in polys:
        x, y = poly.exterior.xy
        fig.add_trace(go.Scatter(
            x=list(x),
            y=list(y),
            mode='lines',
            line=dict(color='black', width=0.3),
            opacity=0.2,
            name='Hogares',
            hoverinfo='text',
            text=f'Colonia: {name}, ID {ID}, Grado : {grado}',
            showlegend=False
        ))

# 3. Layout
fig.update_layout(
    title='Concentración Habitacional',
    xaxis_title='Longitude',
    yaxis_title='Latitude',
    width=700,
    height=900,
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
)

fig.show()


### [Deprecated] Zonas que definen la concentración de valores unitarios promedio del suelo habitacional
Info : 
* Expresado en pesos mexicanos

In [ ]:
# data_zonas_valor

### SHF

Históricos mensuales sobre la información de datos abiertos del Índice SHF de Precios de la Vivienda. Documentos de apoyo sobre los reportes mensuales de las condiciones del mercado de créditos garantizados con garantía hipotecaria.

El índice mide la evolución de los precios de compra-venta de viviendas residenciales en el país, calculándose a partir de datos de avalúos y creditos hipotecarios, proporcionando información sobre la variación de los precios de la vivienda a nivel nacional.

Info : 
* Los cambios se miden como cambios porcentuales del índice (año con año y con el trimestre correspondiente)
* Año base 2017
* Precio Medio CDMX 2025 T1 : 3,866,210

In [ ]:
print('Alcaldias disponibles :', data_idx_SHF[data_idx_SHF['Estado'] == 'Ciudad de México']['Municipio'].unique())

In [ ]:
data_idx_SHF['Indice'] = data_idx_SHF['Indice'].replace(r",", ".",regex=True)
data_idx_SHF['Indice'] = data_idx_SHF['Indice'].astype(float)

dict_trimestres = {1:'03',
                   2:'06',
                   3:'09',
                   4:'12'}


data_idx_SHF['fecha'] = data_idx_SHF.apply(lambda x : str(x['Año']) + '-' + dict_trimestres[int(x['Trimestre'])], axis = 1)
data_idx_SHF['fecha'] = pd.to_datetime(data_idx_SHF['fecha'])

In [ ]:
data_idx_SHF[(data_idx_SHF['Global'] == 'ZM Valle México') & (data_idx_SHF['Año'] >= 2024) & (data_idx_SHF['Trimestre'] == 1)]['Indice'].pct_change()*100

In [ ]:
data_idx_SHF_cdmx = data_idx_SHF[(data_idx_SHF['Estado'] == 'Ciudad de México')]

data_idx_SHF_cdmx_tot = data_idx_SHF_cdmx[data_idx_SHF_cdmx['Municipio'].isna()]
data_idx_SHF_cdmx_gam = data_idx_SHF_cdmx[data_idx_SHF_cdmx['Municipio'] == 'Gustavo A. Madero']
data_idx_SHF_cdmx_izt = data_idx_SHF_cdmx[data_idx_SHF_cdmx['Municipio'] == 'Iztapalapa']
data_idx_SHF_cdmx_bj = data_idx_SHF_cdmx[data_idx_SHF_cdmx['Municipio'] == 'Benito Juárez']
data_idx_SHF_cdmx_cua = data_idx_SHF_cdmx[data_idx_SHF_cdmx['Municipio'] == 'Cuauhtémoc']
data_idx_SHF_cdmx_mh = data_idx_SHF_cdmx[data_idx_SHF_cdmx['Municipio'] == 'Miguel Hidalgo']

In [ ]:
# Create empty figure
fig = go.Figure()

# Add total CDMX line
fig.add_trace(go.Scatter(
    x=data_idx_SHF_cdmx_tot['fecha'],
    y=data_idx_SHF_cdmx_tot['Indice'],
    mode='lines+markers',
    name='CDMX Total'
))

# Add Gustavo A. Madero line
fig.add_trace(go.Scatter(
    x=data_idx_SHF_cdmx_gam['fecha'],
    y=data_idx_SHF_cdmx_gam['Indice'],
    mode='lines+markers',
    name='Gustavo A. Madero'
))

# Add Gustavo A. Madero line
fig.add_trace(go.Scatter(
    x=data_idx_SHF_cdmx_izt['fecha'],
    y=data_idx_SHF_cdmx_izt['Indice'],
    mode='lines+markers',
    name='Iztapalapa'
))

# Add Gustavo A. Madero line
fig.add_trace(go.Scatter(
    x=data_idx_SHF_cdmx_bj['fecha'],
    y=data_idx_SHF_cdmx_bj['Indice'],
    mode='lines+markers',
    name='Benito Juárez'
))

# Add Gustavo A. Madero line
fig.add_trace(go.Scatter(
    x=data_idx_SHF_cdmx_cua['fecha'],
    y=data_idx_SHF_cdmx_cua['Indice'],
    mode='lines+markers',
    name='Cuauhtémoc'
))

# Add Gustavo A. Madero line
fig.add_trace(go.Scatter(
    x=data_idx_SHF_cdmx_mh['fecha'],
    y=data_idx_SHF_cdmx_mh['Indice'],
    mode='lines+markers',
    name='Miguel Hidalgo'
))

# Customize layout
fig.update_layout(
    title='Índice SHF de Precios de la Vivienda (Año base 2017) - Precio Medio CDMX Total : 3866210',
    xaxis_title='Fecha',
    yaxis_title='Índice',
    width=1550,
    height=500,
    template='plotly_white'
)

# Format x-axis
fig.update_xaxes(
    # range=[data_idx_SHF_cdmx_tot['fecha'].min(), data_idx_SHF_cdmx_tot['fecha'].max()],
    tickformat="%Y-%m",
    dtick="M12",
    tickangle=45
)

fig.show()

In [ ]:
def calculo_cdmx(indice):
    total_25_1 = 3866210
    idx_25_1 = 167.58
    return indice * total_25_1 / idx_25_1

calculo_cdmx(100)

* Crecimiento (aceleración y desaceleración)

In [ ]:
df_pct = []
for trimestre in data_idx_SHF_cdmx['Trimestre'].unique():
    for muni in data_idx_SHF_cdmx['Municipio'].unique():
        if str(muni) == 'nan':        
            temp = data_idx_SHF_cdmx[(data_idx_SHF_cdmx['Trimestre'] == trimestre) & (data_idx_SHF_cdmx['Municipio'].isna())]
        else :
            temp = data_idx_SHF_cdmx[(data_idx_SHF_cdmx['Trimestre'] == trimestre) & (data_idx_SHF_cdmx['Municipio'] == muni)]
        temp['Indice_delta_pct'] = temp['Indice'].pct_change()
        df_pct.append(temp)
df_pct = pd.concat(df_pct, axis = 0)
df_pct = df_pct.sort_values(by=['Consecutivo'], ascending=True)

df_pct['Indice_delta_pct'] = df_pct['Indice_delta_pct']*100

df_pct_cdmx_tot = df_pct[df_pct['Municipio'].isna()]
df_pct_cdmx_gam = df_pct[df_pct['Municipio'] == 'Gustavo A. Madero']
df_pct_cdmx_izt = df_pct[df_pct['Municipio'] == 'Iztapalapa']
df_pct_cdmx_bj = df_pct[df_pct['Municipio'] == 'Benito Juárez']
df_pct_cdmx_cua = df_pct[df_pct['Municipio'] == 'Cuauhtémoc']
df_pct_cdmx_mh = df_pct[df_pct['Municipio'] == 'Miguel Hidalgo']

In [ ]:
# Create empty figure
fig = go.Figure()

# Add total CDMX line
fig.add_trace(go.Scatter(
    x=df_pct_cdmx_tot['fecha'],
    y=df_pct_cdmx_tot['Indice_delta_pct'],
    mode='lines+markers',
    name='CDMX Total'
))

# Add Gustavo A. Madero line
fig.add_trace(go.Scatter(
    x=df_pct_cdmx_gam['fecha'],
    y=df_pct_cdmx_gam['Indice_delta_pct'],
    mode='lines+markers',
    name='Gustavo A. Madero'
))

# Add Gustavo A. Madero line
fig.add_trace(go.Scatter(
    x=df_pct_cdmx_izt['fecha'],
    y=df_pct_cdmx_izt['Indice_delta_pct'],
    mode='lines+markers',
    name='Iztapalapa'
))

# Add Gustavo A. Madero line
fig.add_trace(go.Scatter(
    x=df_pct_cdmx_bj['fecha'],
    y=df_pct_cdmx_bj['Indice_delta_pct'],
    mode='lines+markers',
    name='Benito Juárez'
))

# Add Gustavo A. Madero line
fig.add_trace(go.Scatter(
    x=df_pct_cdmx_cua['fecha'],
    y=df_pct_cdmx_cua['Indice_delta_pct'],
    mode='lines+markers',
    name='Cuauhtémoc'
))

# Add Gustavo A. Madero line
fig.add_trace(go.Scatter(
    x=df_pct_cdmx_mh['fecha'],
    y=df_pct_cdmx_mh['Indice_delta_pct'],
    mode='lines+markers',
    name='Miguel Hidalgo'
))

# Customize layout
fig.update_layout(
    title='Cambios porcentuales en el índice SHF de Precios de la Vivienda',
    xaxis_title='Fecha',
    yaxis_title='Índice',
    width=1550,
    height=500,
    template='plotly_white'
)

# Format x-axis
fig.update_xaxes(
    # range=[data_idx_SHF_cdmx_tot['fecha'].min(), data_idx_SHF_cdmx_tot['fecha'].max()],
    tickformat="%Y-%m",
    dtick="M12",
    tickangle=45
)

fig.show()

### Tasa de Crecimiento Poblacional

Contiene la tasa de crecimiento media anual 1990-2020 por alcaldía de la Ciudad de México

In [ ]:
fig = go.Figure()

for alcaldia in data_tasa_crecimiento_alcaldia['Alcaldía'].unique():
    temp = data_tasa_crecimiento_alcaldia[data_tasa_crecimiento_alcaldia['Alcaldía'] == alcaldia]
    temp = temp.iloc[:-1]
    # Add total CDMX line
    fig.add_trace(go.Scatter(
        x=temp['Año final'],
        y=temp['Tasa de crecimiento medio anual'],
        mode='lines+markers',
        name=alcaldia
    ))
    
# Customize layout
fig.update_layout(
    title='Cambios en la tasa de crecimiento medio anual de la población',
    xaxis_title='Fecha',
    yaxis_title='Índice',
    width=1550,
    height=500,
    template='plotly_white'
)

# Format x-axis
fig.update_xaxes(
    # range=[data_idx_SHF_cdmx_tot['fecha'].min(), data_idx_SHF_cdmx_tot['fecha'].max()],
    tickformat="%Y-%m",
    dtick="M12",
    tickangle=45
)

fig.show()

# Análisis de la concentración habitacional, el consumo de agua y la escasez

### Data Set Construction

In [ ]:
def remove_accents(text):
    if not isinstance(text, str):
        return text  # safeguard for non-string input
    return ''.join(
        c for c in unicodedata.normalize('NFD', text)
        if unicodedata.category(c) != 'Mn'
    )
    
def jaccard_similarity_chars(a, b):
    a_set = set(a.lower())
    b_set = set(b.lower())
    intersection = a_set & b_set
    union = a_set | b_set
    if not union:
        return 0.0
    return len(intersection) / len(union)


def jaccard_similarity_tokens(a, b):
    a_tokens = set(remove_accents(a.lower()).split())
    b_tokens = set(remove_accents(b.lower()).split())
    intersection = a_tokens & b_tokens
    union = a_tokens | b_tokens
    if not union:
        return 0.0
    return len(intersection) / len(union)


def match_colonias(df1, df2, col1='colonia', col2='NOMBRE', alc1= 'alcaldia', alc2 = 'DELEGACIO', threshold=0.7):
    matches = []
    for idx1, data in tqdm(df1.iterrows()):
        county = data[0]
        name1 = data[1]
        best_score = 0
        best_match = None
        for name2 in df2[df2[alc2] == county][col2]:    
            score1 = jaccard_similarity_chars(remove_accents(name1.lower()), remove_accents(name2.lower()))
            score2 = jaccard_similarity_tokens(remove_accents(name1.lower()), remove_accents(name2.lower()))

            if score1 > best_score and score2 > 0.3 :
                best_score = score1
                best_match = name2
            if score1 > 0.9 and score2 > 0.3 :
                break
        if best_score >= threshold:
            matches.append((idx1, name1, best_match, best_score))
    return pd.DataFrame(matches, columns=['idx_df1', col1, f'{col2}_matched', 'similarity'])    

In [ ]:
data_to_merge = data_feasibility[['NOMBRE', 'DELEGACIO', 'fact_hidr']]
data_to_merge['DELEGACIO'] = data_to_merge['DELEGACIO'].apply(lambda x : remove_accents(x))
data_to_merge = data_to_merge[~data_to_merge['NOMBRE'].isna()]

In [ ]:
consumo_hogares = pd.merge(hab_consumption, data_hogares_col[['cve_col', 'VivHab2010', 'VivHab2020', 'Area_ha', 'DenViv10', 'DenViv20', 'Sum_TotHog']], on='cve_col', how='left')
consumo_hogares = consumo_hogares[~consumo_hogares['colonia'].isna()]

* Now merge consumo_hogares con data_to_merge

In [ ]:
t = pd.merge(consumo_hogares, data_to_merge, left_on=['alcaldia', 'colonia'], right_on=['DELEGACIO','NOMBRE'], how='left')
still_need_match = t[t['fact_hidr'].isna()][['alcaldia', 'colonia']]

In [ ]:
matched = match_colonias(still_need_match, data_to_merge)

consumo_hogares = pd.merge(consumo_hogares, matched.iloc[:,1:], on='colonia', how = 'left')
consumo_hogares.loc[consumo_hogares['NOMBRE_matched'].isna(), 'NOMBRE_matched'] = consumo_hogares.loc[consumo_hogares['NOMBRE_matched'].isna(), 'colonia']
consumo_hogares = consumo_hogares[~consumo_hogares[['alcaldia', 'colonia']].duplicated()]

consumo_hogares_all = pd.merge(consumo_hogares, data_to_merge, left_on=['alcaldia', 'NOMBRE_matched'], right_on=['DELEGACIO','NOMBRE'], how='left')
consumo_hogares_all = consumo_hogares_all[~consumo_hogares_all[['alcaldia', 'colonia']].duplicated()]
consumo_hogares_all['fact_hidr'] = consumo_hogares_all['fact_hidr'].fillna('NA')

to_num_cat = {
    'NA' : 0,     
    'VERDE' : 1, 
    'AMARILLO' : 2, 
    'NARANJA' : 3,
    'ROJO' : 4 
}
consumo_hogares_all['fact_hidr_cat'] = consumo_hogares_all['fact_hidr'].apply(lambda x : to_num_cat[x])

final_reg = consumo_hogares_all[['alcaldia', 'colonia', 'SUM_cons_t', 'VivHab2020', 'Area_ha', 'DenViv20', 'fact_hidr_cat']]

### Análisis de Correlación

In [ ]:
# Correlation Matrix
num_vars = final_reg.iloc[:,2:]
num_vars = num_vars[num_vars['SUM_cons_t']>0]

correlation_matrix = num_vars.corr(method='pearson')  # Pearson es el default

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='Blues', center=0)
plt.title('Matriz de correlación')
plt.show()

In [ ]:
# Quitar duplicados y 1.0's de la diagonal
corr_pairs = correlation_matrix.unstack()
corr_pairs = corr_pairs[corr_pairs != 1.0]
high_corr = corr_pairs[abs(corr_pairs) > 0.6].sort_values(ascending=False)
print(high_corr)

In [ ]:
sns.pairplot(num_vars)
plt.show()

### Reducción de dimensionalidad

In [ ]:
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:

X = num_vars.loc[:,['VivHab2020']]
y = .iloc[:,:1]

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# # # Apply PCA
# pca = PCA(n_components=1)  # Keep 95% of variance
# X_pca = pca.fit_transform(X_scaled)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
X_scaled

In [ ]:
model = KernelRidge(kernel='rbf', alpha=1.0, gamma=0.1)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [ ]:
plt.figure(figsize=(6, 6))
plt.scatter(y_test, y_pred, alpha=0.7)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], '--', color='black')
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Predicted vs Actual')
plt.grid(True)
plt.show()